In [6]:
def extract_actions(single_history):
    """
    Takes a single string of history (no /) and 
    returns a list of actions in a form such as
    ['c', '1r', '10r', 'f']
    """
    actions = []
    current_action = ''
    for item in single_history:
        if item in ['a', 'c', 'f']:
            actions.append(item)
            current_action = ''
        elif item.isdigit():
            current_action += item
        elif item == 'r':
            actions.append(current_action + item)
            current_action = ''
    return actions
            

def player_money_bet(action_history):
    """
    Returns the amount of money p1 and p2 have bet
    (not including the antee) with the format of p1, p2.
    Assumes that player 1 always moves first for each betting round.
    """
    p1_commited = 0
    p2_commited = 0
    for history in action_history:
        print(history)
        p1_temp = 0
        p2_temp = 0
        for key, action in enumerate(extract_actions(history)):
            if key % 2 == 0:
                if action == 'c':
                    p1_temp = p2_temp
                elif 'r' in action:
                    p1_temp = p2_temp + int(action.replace('r', ''))
                elif action == 'a':
                    p1_temp += 1
            else:
                if action == 'c':
                    p2_temp = p1_temp
                elif 'r' in action:
                    p2_temp = p1_temp + int(action.replace('r', ''))
                elif action == 'a':
                    p2_temp += 1
        print(p1_temp, p2_temp)
        p1_commited += p1_temp
        p2_commited += p2_temp
    
    return p1_commited, p2_commited

def valid_actions(infoset_key, max_bet):
    """
    Returns a list of valid actions based off the tree history.
    """
    player_id = infoset_key.split(";")[0]
    action_history = infoset_key.split(";")[-1].split('/')

    p1_commited, p2_commited = player_money_bet(action_history)
    if p1_commited > max_bet or p2_commited > max_bet:
        error_msg = f"P1 or P2 have bet too much money! p1_commited: {p1_commited} and p2_commited: {p2_commited}"
        error_msg += f" with a max bet of {max_bet}"
        raise ValueError(error_msg)

    print(p1_commited, p2_commited)
    actions = ['c']
    if player_id == "P1":
        if p2_commited > p1_commited: # Being raised against
            actions = ['f'] + actions
        for bet_amount in range(1, max_bet + 1 - p2_commited):
            actions.append(str(bet_amount) + 'r')

    if player_id == "P2":
        if p1_commited > p2_commited: # Being raised against
            actions = ['f'] + actions
        for bet_amount in range(1, max_bet + 1 - p1_commited):
            actions.append(str(bet_amount) + 'r')

    return actions

class Card:
    SUIT_TO_STRING = {
        1: "s",
        2: "h",
        3: "d",
        4: "c"
    }
    
    RANK_TO_STRING = {
        2: "2",
        3: "3",
        4: "4",
        5: "5",
        6: "6",
        7: "7",
        8: "8",
        9: "9",
        10: "T",
        11: "J",
        12: "Q",
        13: "K",
        14: "A"
    }

    RANK_JACK = 11
    RANK_QUEEN = 12
    RANK_KING = 13
    RANK_ACE = 14
    
    STRING_TO_SUIT = dict([(v, k) for k, v in SUIT_TO_STRING.items()])
    STRING_TO_RANK = dict([(v, k) for k, v in RANK_TO_STRING.items()])

    def __init__(self, rank, suit):
        """
        Create a card. Rank is 2-14, representing 2 through Ace,
        while suit is 1-4 representing spades, hearts, diamonds, clubs
        """
        self.rank = rank
        self.suit = suit

    def __repr__(self):
        return "(%s, %s)" % (self.rank, self.suit)
    
    def pretty_repr(self):
        return "%s%s" % (self.RANK_TO_STRING[self.rank], self.SUIT_TO_STRING[self.suit])
    
    def __eq__(self, other):
        return (isinstance(other, self.__class__) and self.rank == other.rank and self.suit == other.suit)
    
    def __hash__(self):
        return hash((self.rank, self.suit))

def evaluate_winner_ac(cards_1, cards_2, community_cards, history):
    """
    This function returns 1, 0, or -1 for player winning,
    tieing, or losing.
    """
    
    print(",".join([x.pretty_repr() for x in cards_1]))
    print(",".join([x.pretty_repr() for x in cards_2]))
    print(",".join([x.pretty_repr() for x in community_cards]))

    # Check for folding
    final_history = history[-1]
    if len(final_history) > 0 and final_history[-1] == 'f':
        if len(final_history) % 2 == 0:
            return 1
        else:
            return 2

    def get_best_card(cards):
        best_card = Card(-1, -1)
        for card in cards:
            if card.rank > best_card.rank:
                best_card = card
            elif card.rank == best_card.rank and card.suit > best_card.suit:
                best_card = card
        return best_card

    # Assuming no one folded, evaluate based on high card
    p1_best_card = get_best_card(cards_1)
    p2_best_card = get_best_card(cards_2)

    print("p1_best_card", p1_best_card)
    print("p2_best_card", p2_best_card)
    
    if p1_best_card.rank > p2_best_card.rank:
        return 1
    elif p2_best_card.rank > p1_best_card.rank:
        return 2
    elif p1_best_card.rank == p2_best_card.rank:
        if p1_best_card.suit > p2_best_card.suit:
            return 1
        else:
            return 2

def terminal_util(infoset_key, p1_cards, p2_cards, co_cards, player_id):
    """
    Returns the utility of a terminal history.
    All action has finished, payouts are determined.
    """

    action_history = infoset_key.split(";")[-1].split('/')

    p1_commited, p2_commited = player_money_bet(action_history)
    winner = evaluate_winner(p1_cards, p2_cards, co_cards, action_history)

    if winner == -1: # Tie
        return 0

    if player_id == "P1":
        if winner == 1:
            return p2_commited
        else:
            return -1 * p1_commited
    if player_id == "P2":
        if winner == 2:
            return p1_commited
        else:
            return -1 * p2_commited

def estimate_hand_strength(player_cards, co_cards, deck):
    simulation_results = []

    for i in range(_NB_SIMULATION):
        opponents_cards, new_deck = deck.draw_random_cards(2)
        new_community_cards, final_deck = new_deck.draw_random_cards(5 - len(co_cards))
        community_cards = sort_cards(co_cards + new_community_cards)
        dummy_history = ['cc']
        winner = evaluate_winner(player_cards, opponents_cards, community_cards, dummy_history)

        if winner == 1: #player wins
            result = 1
        else:
            result = 0

        simulation_results.append(result)
    average_win_rate = sum(simulation_results) / len(simulation_results)
      
    return average_win_rate

def evaluate_winner(cards_1, cards_2, community_cards, history):
    """
    This function returns 1, 0, or -1 for player winning,
    tieing, or losing.
    """
    
    # Check for folding
    final_history = extract_actions(history[-1])
    if len(final_history) > 0 and final_history[-1] == 'f':
        if len(final_history) % 2 == 0:
            return 1
        else:
            return 2

    def get_best_card(cards):
        best_card = Card(-1, -1)
        for card in cards:
            if card.rank > best_card.rank:
                best_card = card
            elif card.rank == best_card.rank and card.suit > best_card.suit:
                best_card = card
        return best_card

    # Assuming no one folded, evaluate based on high card
    p1_best_card = get_best_card(cards_1)
    p2_best_card = get_best_card(cards_2)
    
    if p1_best_card.rank > p2_best_card.rank:
        return 1
    elif p2_best_card.rank > p1_best_card.rank:
        return 2
    elif p1_best_card.rank == p2_best_card.rank:
        if p1_best_card.suit > p2_best_card.suit:
            return 1
        else:
            return 2

#print(valid_actions("P2;3/4;aa/cc/c1r1r", 3))

# player_id = 'P2'
# p1_cards = (Card(5,1), Card(2,1))
# p2_cards = (Card(4,1), Card(4,2))
# community_cards = [(Card(3,2))]
# action_history = ['1rf']

# print(evaluate_winner(p1_cards, p2_cards, community_cards, action_history))

# print(terminal_util(infoset_key, p1_cards, p2_cards, community_cards, player_id))

1
